# Imports

In [1]:
import numpy as np
import pandas as pd
from math import sqrt
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# Node Class

In [2]:
# Node class to initialise instances of each 
class Node():
    
    def __init__(self, feature = None, limit = None, leftSide = None, rightSide = None, gain = None, leaf = None):
        
        self.feature = feature
        self.limit = limit
        self.leftSide = leftSide
        self.rightSide = rightSide
        self.gain = gain
        self.leaf = leaf 

# Decision Tree Regression Algorithm

In [3]:
class decisionTree():
    
    def __init__(self, minSamples, maxDepth):
        self.root = None
        self.minSamples = minSamples
        self.maxDepth = maxDepth
        
    # Compute the information gain for a given split    
    def infoGain(self, parent, leftNode, rightNode):
        # Compute the weight of the left and right node
        leftWeight = len(leftNode) / len(parent)
        rightWeight = len(rightNode) / len(parent)
        
        # Compute the information gain based on the variance of the parent and children nodes
        informationGain = np.var(parent) - (leftWeight * np.var(leftNode) + rightWeight * np.var(rightNode))

        return informationGain
        
    # Split the training data into left and right branches based on a threshold
    def splitTree(self, trainingSet, feature, limit):
        leftBranch = []
        rightBranch = []
        for i in trainingSet:
            if i[feature] <= limit:
                leftBranch.append(i)
            else:
                rightBranch.append(i)
        rightBranch = np.array(rightBranch)
        leftBranch = np.array(leftBranch)
        return leftBranch, rightBranch
    
    # Find the best split for a given feature
    def bestSplit(self, trainingSet, X):
        bestSplitt = {} 
        biggestGain = -1

        # Loop through each feature and each threshold value to find the best spli
        for feature in range(X.shape[1]): 
            featureValues = []
            for i in range(len(trainingSet)):
                featureValues.append(trainingSet[i, feature])
            thresholds = np.unique(featureValues)
            for j in thresholds:
                # Split the node into two sub-trees
                leftSide, rightSide = self.splitTree(trainingSet, feature, j) #splits node into 2 sub-trees
                if (len(leftSide) > 0 and len(rightSide) > 0 ):
                    parent = []
                    for i in range(len(trainingSet)):
                        parent.append(trainingSet[i, -1])

                    leftNode = []
                    for i in range(len(leftSide)):
                        leftNode.append(leftSide[i, -1])
                        
                    rightNode = []
                    for i in range(len(rightSide)):
                        rightNode.append(rightSide[i, -1])

                    currentGain = self.infoGain(parent, leftNode, rightNode) 
                    if currentGain > biggestGain:
                        # Update the best split if the gain is larger than the previous best gain
                        bestSplitt["feature"] = feature
                        bestSplitt["limit"] = j
                        bestSplitt["leftSide"] = leftSide
                        bestSplitt["rightSide"] = rightSide
                        bestSplitt["gain"] = currentGain
                        biggestGain = currentGain
                
        return bestSplitt
   
    # Build the decision tree recursively
    def treeBuild(self, trainingSet, currentDepth = 0):
        
        # Split training data into features and labels
        X = trainingSet[:,:-1] # Everything but the last value
        Y = []
        for i in range(len(trainingSet)):
            Y.append(trainingSet[i, -1]) # only the last value
        
        # Recursively build the tree until the stopping condition is met
        if X.shape[0] >= self.minSamples and currentDepth <= self.maxDepth:
            bestSplitNode = self.bestSplit(trainingSet, X)
            
            if "gain" in bestSplitNode and bestSplitNode["gain"] > 0:
                leftTree = self.treeBuild(bestSplitNode["leftSide"], currentDepth + 1)
                rightTree = self.treeBuild(bestSplitNode["rightSide"], currentDepth + 1)
                node = Node(bestSplitNode["feature"], bestSplitNode["limit"], leftTree, rightTree, bestSplitNode["gain"])

                return node
        
        # Create a leaf node with the average value of the training set
        leafValue = np.mean(Y) #calculates mean of leaf nodes
        val = Node(leaf = leafValue)
        return val
    
    # Recursive function predict the label for a test row
    def predictionLoop(self, testRow, root):
        if root.leaf != None: # Leaf node
            return root.leaf
        
        featureVal = testRow[root.feature]
        if featureVal <= root.limit:
            return self.predictionLoop(testRow, root.leftSide)
        else:
            return self.predictionLoop(testRow, root.rightSide)
        
   
    # Function to predict the labels for a set of test rows
    def predict(self, xTest):
        predictions = []
        for row in xTest:
            predictions.append(self.predictionLoop(row, self.root)) 
        return predictions

        
    # Function to train the decision tree and set the root node
    def fit(self, X, Y):
        trainingSet = np.concatenate((X, Y), axis=1) # Join features and labels
        self.root = self.treeBuild(trainingSet)

# Cleaning and Splitting Data

In [4]:
modelEncoder = LabelEncoder()
transmissionEncoder = LabelEncoder()
fuelTypeEncoder = LabelEncoder()

# Takes given dataset and returns split data
def dataset(brand):

    file = pd.read_csv(brand, quotechar='"', skipinitialspace=True) # Reads the dataset

    # Removes all outliers from the 'year' column
    for i in ['year']:
        q75,q25 = np.percentile(file.loc[:,i],[75,25])
        IQR = q75-q25 # Interquartile range
    
        max = q75+(1.5*IQR)
        min = q25-(1.5*IQR)
    
        file.loc[file[i] < min, i] = np.nan # Replaces outliers smaller than min with NaN
        file.loc[file[i] > max, i] = np.nan # Replaces outliers larger than max with NaN

    file = file.dropna(axis = 0) # Removes rows with NaN values

    # Turns string values into numerical values using LabelEncoder
    modelEncoder.fit(file["model"])
    file["model"] = modelEncoder.transform(file["model"])

    transmissionEncoder.fit(file["transmission"])
    file["transmission"] = transmissionEncoder.transform(file["transmission"])

    fuelTypeEncoder.fit(file["fuelType"])
    file["fuelType"] = fuelTypeEncoder.transform(file["fuelType"])

    file = file.head(10000) # Limits dataset size to 10,000

    X = file.drop(['price'], axis = 1).to_numpy()
    Y = file['price'].values.reshape(-1,1)

    # Splits data into 75% training and 25% testing data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 601)
    
    return  X_train, X_test, Y_train, Y_test, file, X, Y

X_train, X_test, Y_train, Y_test, file, X, Y= dataset("../UKUsedCarDataSet/vw.csv") # Change file name to change dataset here

# Using Scikit-Learn's GridSeacrhCV to find optimal parameters

In [ ]:
myTree = DecisionTreeRegressor(random_state = 601, max_depth = 34, min_samples_split = 3) #, max_depth = 3
myTree.fit(X_train, Y_train)
print("R Squared: ", myTree.score(X_test, Y_test))


params = {'min_samples_split': list(range(2, 100)), 'max_leaf_nodes': list(range(2, 100)), 'max_depth': list(range(2, 100))}
grid_search_cv = GridSearchCV(myTree, params, cv=3)
grid_search_cv.fit(X_train, Y_train)

grid_search_cv.best_estimator_

# RMSE

In [5]:
def rmse(test, pred):
    MSE = np.square(np.subtract(test, pred)).mean()
    return sqrt(MSE)

# Hold-out Validation

In [6]:
def evaluation():   

    # Initialises and trains model
    myTree = decisionTree(3, 34)
    myTree.fit(X_train, Y_train) 
    
    Y_pred = myTree.predict(X_test)
    error = rmse(Y_test, Y_pred) 
    print('The RMSE value is:', round(error, 2))

evaluation()

The RMSE value is: 8200.16


# Cross Validation

In [ ]:
def crossValidation():

    RMSEScores = []

    kf = KFold(n_splits = 5, shuffle = True, random_state = 601)
    
    # Iterate over the KFold splits
    for n, m in kf.split(X):
        X_train, X_test = X[n], X[m]
        Y_train, Y_test = Y[n], Y[m]
        
        # Fit the model on the training data
        dt = decisionTree(3, 34)
        dt.fit(X_train, Y_train)
        
        # Predict on the validation data
        Y_pred = dt.predict(X_test)
        
        # Compute the RMSE score for this fold
        RMSEValue = rmse(Y_test, Y_pred)
        RMSEScores.append(RMSEValue)
    
    # Compute the average RMSE score across all folds
    print("Average RMSE score:", np.mean(RMSEScores))

crossValidation() #1664

# User Input Predictions

In [13]:
inputPred = []
entries = []

def userInput():
    chooseBrand = input("Choose your car brand: Audi, BMW, Ford, Hyundai, Mercedes, Skoda, Toyota, Vauxhall or Volkswagen \n")
    
    match chooseBrand:
        case "Audi":
            return "../UKUsedCarDataSet/audi.csv"
        case "BMW":
            return "../UKUsedCarDataSet/bmw.csv"
        case "Ford":
            return "../UKUsedCarDataSet/ford.csv"
        case "Hyundai":
            return "../UKUsedCarDataSet/hyundi.csv"
        case "Mercedes":
            return "../UKUsedCarDataSet/merc.csv"
        case "Skoda":
            return "../UKUsedCarDataSet/skoda.csv"
        case "Toyota":
            return "../UKUsedCarDataSet/toyota.csv"
        case "Vauxhall":
            return "../UKUsedCarDataSet/vauxhall.csv"
        case "Volkswagen":
            return "../UKUsedCarDataSet/volkswagen.csv"
        case _:
            print("Invalid input")
            return userInput()

X_train, X_test, Y_train, Y_test, file, X, Y = dataset(userInput())

print("\n ***Training Tree Model***")
timer = time.time()

# Initialises and trains model
myTree = decisionTree(3, 93)  
myTree.fit(X_train, Y_train)

print("\n List of models:")
print(list(modelEncoder.classes_))

inputPred.append((modelEncoder.transform([input("\nWhat Model is your car? ")]))[0])
inputPred.append(int(input("What year is your car? ")))
inputPred.append((transmissionEncoder.transform([input("What transmission is your car? ")]))[0])
inputPred.append(int(input("How much mileage does your car have? ")))
inputPred.append((fuelTypeEncoder.transform([input("What's your car fuel type? ")]))[0])
inputPred.append(int(input("How much is your cars tax? ")))
inputPred.append(float(input("What's MPG of your car? ")))
inputPred.append(float(input("What the engine size of your car? ")))
entries.append(inputPred)


print("\n ***Predicting***")
Y_pred = myTree.predict([inputPred]) # Predicts price of car
print("\n Predicted price for your car is: £", round(Y_pred[0], 2))
print("\n ***Predicted in", time.time() - timer,"seconds***")


 ***Training Tree Model***

 List of models:
['A1', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'Q2', 'Q3', 'Q5', 'Q7', 'Q8', 'R8', 'RS3', 'RS4', 'RS5', 'RS6', 'RS7', 'S3', 'S4', 'S5', 'S8', 'SQ5', 'SQ7', 'TT']

 ***Predicting***

 Predicted price for your car is: £ 44985.0

 ***Predicted in 286.7857460975647 seconds***
